In [1]:
from my_utils import Utils
import torch
import os
os.environ['NEURITE_BACKEND'] = 'pytorch'
os.environ['VXM_BACKEND'] = 'pytorch'
import voxelmorph as vxm
import matplotlib.pyplot as plt
import glob
import re

In [2]:
original_tissues_path='/nfs2/baos1/rudravg'
round_0_name='GCA112TIA'
save_dir=''
model_path='/home-local/rudravg/test_DAPI/epochs/epoch_50/epoch_50.pth'
#Original Tissue files
search_pattern = os.path.join(original_tissues_path, round_0_name + '*')
matching_files = glob.glob(search_pattern)
def extract_round_number(filename):
    match = re.search(r'_ROUND_(\d+)_', filename)
    if match:
        return int(match.group(1))
    else:
        return 0
sorted_files = sorted(matching_files, key=extract_round_number) # List of paths for stored files.
mask_search_pattern = os.path.join(original_tissues_path, 'Retention_Masks', round_0_name + '*')

In [3]:
mask_files = glob.glob(mask_search_pattern)
if mask_files:
    mask_name = mask_files[0]
else:
    mask_name = None

In [4]:
original_tissue=sorted_files[0]
target_tissue=sorted_files[1]
fixed,moving=Utils.load_images_and_apply_mask(original_tissue,target_tissue,mask_name)

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model,device=Utils.load_model(model_path)

/home/local/VANDERBILT/rudravg/miniconda3/envs/dapi_reg/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
registered_tissue=Utils.register_tissues(moving,fixed,model,device)

In [7]:
after_ncc=Utils.calculate_ncc(fixed.ravel(),registered_tissue.ravel())
before_ncc=Utils.calculate_ncc(fixed.ravel(),moving.ravel())
print('NCC between moving and recon:',before_ncc)
print('NCC between fixed and recon:',after_ncc)